In [18]:
from opentrons import protocol_api
import socket
from opentrons.types import *
import opentrons.execute

def run(protocol: protocol_api.ProtocolContext):
    # Load labware and pipette
    tiprack = protocol.load_labware('opentron_96_tiprack_300ul', '11', '300ul Tip Rack')
    pipette = protocol.load_instrument('p300_single', mount = 'left', tip_racks=[tiprack])
    protocol.home()  # Initial homing

    # Socket server setup
    host = '0.0.0.0'
    port = 5000
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(1)
    print(f"Server listening on {host}:{port}")

    try:
        conn, addr = server_socket.accept()
        print(f"Connected by {addr}")
        with conn:
            while True:
                data = conn.recv(1024).decode('utf-8').strip()
                if not data:
                    break
                print(f"Received command: {data}")

                # Process command
                response = "Unknown command"
                if data == 'HOME':
                    protocol.home()
                    response = "Homed"
                elif data.startswith('MOVE_TO_COORDS'):
                    _, x, y, z = data.split()
                    location = Location(Point(float(x), float(y), float(z)), None)
                    pipette.move_to(location)
                    response = f"Moved to ({x}, {y}, {z})"
                elif data.startswith('MOVE_TO_WELL'):
                    _, well_name = data.split()
                    well = tiprack.wells_by_name()[well_name]
                    pipette.move_to(well.top())
                    response = f"Moved to {well_name}"
                elif data == 'EXIT':
                    break
    except Exception as e:
        response = f"Error: {str(e)}"
    finally:
        pass
    return

protocol = opentrons.execute.get_protocol_api("2.19")
protocol.home()    
run(protocol)                        

Execution of a86ac15f-e4ba-4b93-9ea6-1a1f5d5ee396 failed
Traceback (most recent call last):
  File "/usr/lib/python3.10/site-packages/opentrons/protocol_engine/state/labware.py", line 427, in get_definition_by_uri
KeyError: 'opentrons/opentron_96_tiprack_300ul/1'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/lib/python3.10/site-packages/opentrons/protocol_engine/execution/equipment.py", line 163, in load_labware
  File "/usr/lib/python3.10/site-packages/opentrons/protocol_engine/state/labware.py", line 429, in get_definition_by_uri
opentrons.protocol_engine.errors.exceptions.LabwareDefinitionDoesNotExistError: Error 4000 GENERAL_ERROR (LabwareDefinitionDoesNotExistError): Labware definition for matching opentrons/opentron_96_tiprack_300ul/1 not found.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/site-packages/opentrons/protocol

ProtocolCommandFailedError: Error 4000 GENERAL_ERROR (ProtocolCommandFailedError): PythonException: FileNotFoundError: Labware "opentron_96_tiprack_300ul" not found with version 1 in namespace "opentrons".

In [17]:
from opentrons import protocol_api
import opentrons.execute

from opentrons.types import *

metadata = {
    'protocolName': 'Simple Movement Test',
    'author': 'Your Name',
    'description': ('Moves the pipette to an absolute coordinate (150,150,150) '
                    'and then to tip rack well A2 in slot 11.'),
    'apiLevel': '2.9'
}

def run(protocol: protocol_api.ProtocolContext):
    # Load a tip rack in slot 11
    tiprack = protocol.load_labware('opentrons_96_tiprack_300ul', '11', '300ul Tip Rack')
    
    # Load a single-channel P300 pipette on the right mount
    pipette = protocol.load_instrument('p300_single', mount='left', tip_racks=[tiprack])
    
    well = tiprack.wells_by_name()['A1']
    
    # Move to an absolute coordinate (150,150,150) relative to the deck
    target_point = Point(150, 150, 150)
    target_location = Location(target_point, well)
    pipette.move_to(target_location)
    
    # Move to tip rack well A2
    #pipette.move_to(tiprack['A2'])

protocol = opentrons.execute.get_protocol_api("2.19")
protocol.home()    
run(protocol)

In [19]:
import socket

# Server configuration
SERVER_IP = "0.0.0.0"  # Listen on all interfaces of the Pi
SERVER_PORT = 5000     # Arbitrary choice of an open port

# Create a socket object
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# Bind the socket to an address/port
server_socket.bind((SERVER_IP, SERVER_PORT))
# Listen for incoming connections (up to 1 backlog)
server_socket.listen(1)

print(f"Server listening on {SERVER_IP}:{SERVER_PORT}")

try:
    # Accept a single incoming connection
    conn, addr = server_socket.accept()
    print(f"Connected by {addr}")

    # Receive data (up to 1024 bytes) from the client
    data = conn.recv(1024).decode('utf-8')
    print("Data received from client:", data)
    
    # Here you could parse the data (e.g. which well to pipette to) and then
    # control your Opentrons pipette accordingly:
    #
    # Example:
    # if data == "well A1":
    #     # Perform pipetting action in well A1
    #     protocol.pick_up_tip()
    #     protocol.aspirate(50, reagent)
    #     protocol.dispense(50, plate['A1'])
    #     protocol.drop_tip()

    # Send a response to the client (optional)
    response = "Message received!"
    conn.sendall(response.encode('utf-8'))

    conn.close()
except KeyboardInterrupt:
    pass
finally:
    server_socket.close()
    print("Server socket closed.")

Server listening on 0.0.0.0:5000
Connected by ('192.168.13.131', 64727)
Data received from client: well A1
Server socket closed.
